In [18]:
%pip install pandas numpy tqdm scikit-learn

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 4.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: /home/morg/python-envs/HW3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
import scipy.sparse as sp
import numpy as np
import sys
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score

In [3]:
df = pd.read_csv('./Data/23-24-problem_logs.csv')

In [6]:
df.sort_values(by=['user_xid','skill_id','start_time'], inplace=True)

In [7]:
wins = []
fails = []
for name, group in tqdm(df.groupby(by=['user_xid', 'skill_id'])):
    w = 0
    f = 0
    for idx, row in group.iterrows():
        wins.append(w)
        fails.append(f)
        if row['discrete_score'] == 1:
            w += 1
        else:
            f += 1
             
    

100%|██████████| 1424943/1424943 [09:32<00:00, 2491.14it/s]


In [8]:
df['wins'] = wins
df['fails'] = fails

In [10]:
skills = np.sort(df.skill_id.unique())
n_s = len(skills)

In [11]:
res_arr = sp.dok_array((df.shape[0], 3*n_s))
i = 0
for idx, row in tqdm(df.iterrows()):
    #user_idx = np.where(users == row['user_xid'])[0][0]
    skill_idx = np.where(skills == row['skill_id'])[0][0]
    # res_arr[i,user_idx] = 1
    # res_arr[i,problem_idx] = 1
    res_arr[i,skill_idx] = 1
    res_arr[i,n_s+skill_idx] = row['wins']
    res_arr[i,2*n_s+skill_idx] = row['fails']
    i += 1

12075734it [21:26, 9387.20it/s] 


In [13]:
res_arr_coo = res_arr.tocoo()
y = df.discrete_score.to_numpy(copy=True)
fn = './Data/23-24-problem_logs'
fn_X = fn + '_X.npz'
fn_y = fn + '_y.npy'
sp.save_npz(fn_X, res_arr_coo)
np.save(fn_y, y)

In [32]:
model = LogisticRegression(penalty=None, solver='lbfgs',verbose=1, max_iter=10**4)

In [33]:
model.fit(res_arr_coo,y)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         1576     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  2.38392D+00


 This problem is unconstrained.



At iterate   50    f=  6.14771D-01    |proj g|=  3.83529D-02

At iterate  100    f=  6.07727D-01    |proj g|=  4.41595D-02

At iterate  150    f=  6.03632D-01    |proj g|=  9.03428D-02

At iterate  200    f=  6.01395D-01    |proj g|=  1.02215D-02

At iterate  250    f=  5.99877D-01    |proj g|=  1.48832D-02

At iterate  300    f=  5.98919D-01    |proj g|=  3.97702D-02

At iterate  350    f=  5.97892D-01    |proj g|=  1.60127D-02

At iterate  400    f=  5.97096D-01    |proj g|=  1.16112D-02

At iterate  450    f=  5.96415D-01    |proj g|=  2.07394D-02

At iterate  500    f=  5.95790D-01    |proj g|=  1.31722D-02

At iterate  550    f=  5.95364D-01    |proj g|=  6.04595D-02

At iterate  600    f=  5.95054D-01    |proj g|=  8.39199D-03

At iterate  650    f=  5.94698D-01    |proj g|=  5.62741D-02

At iterate  700    f=  5.94442D-01    |proj g|=  9.32168D-03

At iterate  750    f=  5.94240D-01    |proj g|=  8.11093D-03

At iterate  800    f=  5.94089D-01    |proj g|=  8.67286D-03

At iter

LogisticRegression(max_iter=10000, penalty=None, verbose=1)

In [34]:
y_pred = model.predict_proba(res_arr_coo)[:,1]

In [35]:
log_loss(y,y_pred)

0.592389595850502

In [36]:
roc_auc_score(y,y_pred)

0.6889990501918849